In [1]:
%%capture
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [3]:
df = df.drop(columns = 'Id')
test =test.drop(columns = 'Id')

In [4]:
X = df.drop(columns = 'Pawpularity')
y = df.Pawpularity

In [5]:
#create_folds.py 
from sklearn import model_selection
df["kfold"] = -1

df = df.sample(frac=1).reset_index(drop=True)

kf = model_selection.StratifiedKFold(n_splits=5, shuffle=False, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(X=df,y=df.Pawpularity.values)):
    print(len(train_idx), len(val_idx))
    df.loc[val_idx, 'kfold'] = fold


7929 1983
7929 1983
7930 1982
7930 1982
7930 1982


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


In [6]:
import joblib
import lightgbm as lgb
import xgboost as xg
import pandas as pd
from sklearn import metrics


from sklearn import tree
def run(fold):
    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)
    df_train = df_train.drop(columns = 'kfold')
    df_valid = df_valid.drop(columns = 'kfold')
    x_train = df_train.drop('Pawpularity', axis=1).values
    y_train = df_train.Pawpularity.values
    x_valid = df_valid.drop('Pawpularity', axis=1).values
    y_valid = df_valid.Pawpularity.values
    model = xg.XGBRegressor(tree_method='gpu_hist')
    model.fit(x_train, y_train)
    y_pred = model.predict(x_valid)
    accuracy_score = metrics.mean_absolute_error(y_valid,y_pred)
    print(f"Fold={fold}, accuracy_score={accuracy_score}")
    File_name = 'model_' + str(fold)
    joblib.dump(
    model,File_name)
for i in range(5):
    run(fold = i)

Fold=0, accuracy_score=15.76675484340358
Fold=1, accuracy_score=15.632957846602586
Fold=2, accuracy_score=15.658862034199096
Fold=3, accuracy_score=15.50946611737387
Fold=4, accuracy_score=15.57094066841932


In [7]:
model_0 = joblib.load('./model_0')
model_1 = joblib.load('./model_1')
model_2 = joblib.load('./model_2')
model_3 = joblib.load('./model_3')
model_4 = joblib.load('./model_4')

In [8]:
y_final_3 = model_3.predict(test)
y_final_0 = model_0.predict(test)
y_final_1 = model_1.predict(test)
y_final_2 = model_2.predict(test)
y_final_4 = model_4.predict(test)

In [9]:
y_final_avg = (y_final_0 + y_final_1 +y_final_2 + y_final_3 + y_final_4)/5

#####  IMAGE ONE

In [10]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import os
import torch
import torchvision
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import albumentations

In [11]:
import torch
import torch.nn as nn
from skimage import io, transform

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,image_path,features,targets,augmentations=None):
        self.image_path = image_path
        self.features = features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self,item):
        image = io.imread(self.image_path[item])
        features = self.features[item,:]
        targets = self.targets[item]
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }

In [12]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [13]:
import torch
import torch.nn as nn

model_name = 'swin_tiny_patch4_window7_224'

out_dim    = 1

class get_model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        self.model.head = nn.Sequential(nn.Linear(self.model.head.in_features,512),
                                              nn.Linear(512,256))
        self.last = nn.Linear(256 + 12, 128)
        self.depth1 = nn.Linear(128,64)
        self.depth2 = nn.Linear(64,1)
    def forward(self, image, features):
        x = self.model(image)
        x = self.last(torch.cat([x, features], dim=1))
        x = self.depth1(x)
        x = self.depth2(x)
        return x

In [14]:
import os
from sklearn import metrics
import albumentations
device = 'cuda'
epochs = 12
data_path = '../input/petfinder-pawpularity-score'

valid_aug = albumentations.Compose(
    [
        albumentations.Resize(224, 224, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [15]:
feats = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
    'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
]

In [16]:
from itertools import chain
device = 'cuda'
model_f = get_model()
model_f.to(device)
df_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
for e in {0,1,3}:
    
    model_f.load_state_dict(torch.load('../input/classifier-of-pawpularity-of-deep-learning/model-epoch'+str(e)+'.pth'))
    data_path = '../input/petfinder-pawpularity-score'
    device = 'cuda'
    test_images = df_test.Id.values.tolist()
    test_images = [os.path.join(data_path,'test',i + '.jpg') for i in test_images]

    test_dataset =  CustomDataset(image_path = test_images,features=df_test[feats].values,targets = np.ones(len(test_images)),augmentations=valid_aug)
    test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=False) 


    final_outputs = []

    with torch.no_grad():
        for data in test_loader:
            inputs = data['image']
            features=data['features']
            inputs = inputs.to(device, dtype=torch.float)
            features=features.to(device, dtype=torch.float)
            output = model_f(inputs,features)
            output = (torch.sigmoid(output).detach().cpu().numpy()*100).tolist()
            final_outputs.extend(output)
    final_outputs = list(chain.from_iterable(final_outputs)) 
    y_final_avg = (y_final_avg + final_outputs)/2


submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
# final_outputs = (final_outputs + y_final_avg)/2
submission['Pawpularity'] = y_final_avg
submission.to_csv('submission.csv',index = False)